In [ ]:
import hvplot.pandas
import pandas as pd
import panel as pn
import numpy as np
from bokeh.models.widgets import NumberFormatter, BooleanFormatter
pn.extension()
# Activate the Panel extension for use in a notebook
pn.extension("tabulator")

# Assume 'data.csv' is your dataset path and correctly formatted
#source_data = pd.read_csv('/kaggle/input/target-prioritization/data.csv')
# Sample data definition for demonstration purposes
source_data = pd.read_csv('data.csv')

source_data.rename({'Unnamed: 0':'IDs',
                   'Gene Name or Symbol':'Name',
                   'Product Description':'Desc',
                   'prediction':'Score',
                    'Target':'Validated'} ,axis=1,inplace=True)

hidden_columns = [n for n in source_data.columns if n not in ['IDs', 'Desc', 'Score', 'Name', 'Validated', 'TM', 'Essential'] ]
hidden_columns.append('index')
source_data.reset_index(drop=True,inplace=True)
source_data.sort_values('Score',ascending=False,inplace=True)
source_data['Validated']=source_data['Validated'].astype('bool')
source_data['Score']=source_data['Score'].astype(float).round(2)
source_data['MSF']=source_data['MSF'].astype(float).round(1)
source_data['Structure']=source_data['Structure'].astype(float).round(1)
source_data['Proteomics']=source_data['Proteomics'].astype(float).round(1)
source_data['Druggability']=source_data['Druggability'].astype(float).round(1)
source_data['MIF']=source_data['Druggability'].astype(float).round(1)

print(np.array(hidden_columns))



# Gauge indicator setup
gauge_Score = pn.indicators.Gauge(
    name='Score',
    value=0,
    bounds=(0, 1),
    colors=[(0.3, 'red'), (0.7, 'orange'), (1, 'green')],
    width=200,
    height=200,
    title_size=12
)

gauge_MSF = pn.indicators.Gauge(
    name='MSF',
    value=source_data['MSF'].min(),
    bounds=(source_data['MSF'].min(), source_data['MSF'].max()),
    format='{value}',
    width=200,
    height=200,
    title_size=12
    
)

gauge_MIF = pn.indicators.Gauge(
    name='MIF',
    value=source_data['MIF'].min(),
    bounds=(source_data['MIF'].min(), source_data['MIF'].max()),
    format='{value}',
    width=200,
    height=200,
    title_size=12
)

gauge_Structure = pn.indicators.Gauge(
    name='Structure',
    value=source_data['Structure'].min(),
    bounds=(source_data['Structure'].min(), source_data['Structure'].max()),
    format='{value}',
    width=200,
    height=200,
    title_size=12
)

gauge_Proteomics = pn.indicators.Gauge(
    name='Proteomics',
    value=source_data['Proteomics'].min(),
    bounds=(source_data['Proteomics'].min(), source_data['Proteomics'].max()),
    format='{value}',
    width=200,
    height=200,
    title_size=12
)

gauge_Druggability = pn.indicators.Gauge(
    name='Druggability',
    value=source_data['Druggability'].min(),
    bounds=(source_data['Druggability'].min(), source_data['Druggability'].max()),
    format='0.0',
    width=200,
    height=200,
    title_size=12
)

tabulator_editors = {
    'float': None}


# Formatting for the Tabulator table
bokeh_formatters = {
    #'float': NumberFormatter(format='0.0'),
    #'Score': NumberFormatter(format='0.0%'),
    'Validated': BooleanFormatter(),
    'Essential': BooleanFormatter(),
    #'MSF': NumberFormatter(format='0.0'),
    #'Structure': NumberFormatter(format='0.0'),
}
# Tabulator table setup
table = pn.widgets.Tabulator(
    source_data,
    #layout = 'fit_data_stretch',
    formatters=bokeh_formatters,
    editors={'Validated': 'checkbox',
             'IDs':None,
            'Score':None,
            'Name':None,
            'Desc':None,
            'TM':None,
            'Essential':None},
    selectable=True,
    widths={'TM':60,'Desc':200 },

    name="Table",
    hidden_columns=hidden_columns,
)

# Watch the selection property to update the gauge
def update_gauge(event):
    selected_indices = event.new
    if selected_indices:
        
        gauge_MIF.value = source_data.iloc[selected_indices[0]]['MIF']
        gauge_Score.value = source_data.iloc[selected_indices[0]]['Score']
        gauge_MSF.value = source_data.iloc[selected_indices[0]]['MSF']
        gauge_Structure.value = source_data.iloc[selected_indices[0]]['Structure']
        gauge_Druggability.value = source_data.iloc[selected_indices[0]]['Druggability']
        gauge_Proteomics.value = source_data.iloc[selected_indices[0]]['Proteomics']
    else:
        gauge_Score.value = 0
        gauge_MSF.value = source_data['MSF'].min()
        gauge_Structure.value = source_data['Structure'].min()
        gauge_Druggability.value =  source_data['Druggability'].min()
        gauge_Proteomics.value =  source_data['Proteomics'].min()
        gauge_MIF.value =  source_data['MIF'].min()
        

css = """
.bk-root .bk-gauge-value {
    font-size: 12px; /* Adjust the size as needed */
}
"""
pn.config.raw_css.append(css)

table.param.watch(update_gauge, 'selection')
title = pn.pane.Markdown("# OptimaTarget Dashboard", align='center', width=400)
first_row = pn.Row(gauge_Score, gauge_MSF)
second_row = pn.Row(gauge_MIF, gauge_Structure)
third_row = pn.Row(gauge_Druggability, gauge_Proteomics)

gauge_columns = pn.Column(first_row, second_row, third_row)

layout = pn.Column(title, pn.Row(table, gauge_columns))

#from bokeh.resources import INLINE

layout.servable()

In [4]:
#pn.indicators.Gauge??